# Amazon SageMaker Profiler Example

This notebook will walk you through creating a training job with the profiler feature enabled.

# 1. Install Dependencies


### SageMaker Python SDK and `smdebug`

The first thing you will need to do is install the private beta versions of the SageMaker Python SDK and the `smdebug` library. This will enable you to call a private version of the API that allows you to create SageMaker training jobs with the profiler enabled.

In [ ]:
! pip install ../sdk/sagemaker-1.60.3.dev0.tar.gz -q
! pip install ../sdk/smdebug-0.9.2b20200810-py2.py3-none-any.whl

# The following command will enable the SDK to use new profiler configs in the API
! aws configure add-model --service-model file://../sdk/sagemaker-2017-07-24.normal.json --service-name sagemaker

If you run this notebook in Jupyterlab and not Jupyter, you need to install the jupyterlab extensions 
`@jupyter-widgets/jupyterlab-manager` and `@bokeh/jupyter_bokeh`. We provide a [SageMaker Lifecycle configuration](lifecycle_config/on_start.sh) that automatically installs these extensions when your notebook instance is started. Check out [this blog](https://aws.amazon.com/blogs/machine-learning/customize-your-amazon-sagemaker-notebook-instances-with-lifecycle-configurations-and-the-option-to-disable-internet-access/) for how to create and attach a Lifecycle configuration to your notebook instance.

# 2. Create A Training Job With Profiling Enabled

You will use the standard [SageMaker Estimator API for PyTorch](https://sagemaker.readthedocs.io/en/stable/frameworks/pytorch/sagemaker.pytorch.html) to create training jobs. To create a training job with the profiler enabled, all you need to do is create a `ProfilerConfig` object and pass it into the `profiler_config` parameter of an `Estimator`.


Set profiler configuration

In [ ]:
from sagemaker.profiler import ProfilerConfig 

profiler_config = ProfilerConfig(
    profiling_interval_millis=500,
    profiling_parameters={
        "ProfilerEnabled": str(True),
        "UsePyinstrument": str(True),
        "DetailedProfilingConfig": "{\"StartStep\": 2, \"NumSteps\": 2}"
    }
)

### Set region where this notebook is running

In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

image_name = f'385479125792.dkr.ecr.{region}.amazonaws.com/profiler-gpu:pt_tag1'
print(f"image being used is {image_name}")

### Define PyTorch estimator

In [ ]:
import sagemaker
from sagemaker.pytorch import PyTorch

estimator = PyTorch(
    role=sagemaker.get_execution_role(),
    image_name=image_name,
    train_instance_count=1,
    train_instance_type='ml.p3.8xlarge',
    source_dir='demo',
    entry_point='train_pt.py',
    framework_version='1.5.0',
    profiler_config=profiler_config)

### Start training job

In [ ]:
estimator.fit(wait=False)

# 3.  Analyse profiling data

Run notebook ../profiler/profiler_generic_dashboard.ipynb with training_job_name and region as printed below


In [ ]:
import boto3

session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname:{training_job_name} and region:{region}")
print(f"Run notebook ../profiler/profiler_generic_dashboard.ipynb with parameters training_job_name:{training_job_name} and region:{region} ")